In [7]:
import numpy as np
twodiceprob = {}
dice = [4,6,8,10,12,20]
# Finds the probability in all scenarios with two dice against each other
def findprob(d1, d2):
    # total is the number of outcomes that could could occur
    total = d1 * d2
    # number of successes of d1 vs. d2
    successes = 0
    for i in range(d1):
        for j in range(d2):
            if i < j:
                successes += 1
            elif i == j and d1 > d2:
                successes += 1
    # Gets the total probability of winning
    prob = round(successes/total,5)
    # Gets the points of a success
    points = round(prob * d1/2,5)
    return (prob, points)

# Stores a probability and avg. points for each 
for d1 in dice:
    for d2 in dice:
        if d1 != d2:
            twodiceprob[(d1,d2)] = findprob(d1,d2)

In [8]:
findprob(4,10000)

(0.99975, 1.9995)

In [25]:
### This develops a min and max function, to find the max/min chance of P1 winning
### This is useful in a zero-sum as it leads to an AI maximising the probability of winning
### The functions below enumerate all possible situations in a small game

# Stores the 
max_states = {}
min_states = {}

def checkwhowon(moves):
    # Counts up the scores to see who's won
    total = 0
    for move in moves:
        if move[2] == 1:
            total += move[0]
        else:
            total -= move[1]
            
    if total > 0:
        return 1
    else:
        return 0
        
def get_moveset(moves):
    # Converts moves to tuple to be hashed
    return tuple([tuple(x) for x in moves])

""" Gets the max probability of winning from a particular situation
Parameters: moves so far, the turn the game is on, the probability of the current state
Returns: The largest probability of winning

"""
def max_move(moves, turn, prob):
    # Find the move with the largest probability of winning
    if turn == 7:
        # If game is over, check who's won
        result = checkwhowon(moves)
        moveset = get_moveset(moves)
        max_states[moveset] = [turn, result]
        return result

    # Store the result of largest probability outcome
    # They must be >= 0
    largest = 0
    # Go through all possible dice, and if not illegal - branch off move
    for d1 in dice:
        # If it's the first turn and the dice wasn't used last by the player last round
        if turn == 1 or d1 != moves[-1][0]:
            # Find the avg score
            score = min_move(moves,turn+1,prob,d1)
            largest = max(largest, score) # Update to find largest prob of winning
    
    # Gets moves in a hashable form
    moveset = get_moveset(moves)
    # Stores the probability of winning from this position
    max_states[moveset] = [turn, largest] # Update moveset prob
    # Returns the probability of winning
    return largest

""" Gets the min probability of winning from a particular situation
Parameters: moves so far, the turn the game is on, the probability of the current state, the other dice
Returns: The smallest probability of winning
"""

def min_move(moves, turn, prob, d1):
    
    # Store the smallest probability outcome of winning for P1
    # Must be <= 1
    smallest = 1
    for d2 in dice:
        if  d2 != d1 and (turn == 2 or d2 != moves[-1][1]):

            if moves:
                op = moves + [[d1,d2,1]]
            else:
                op = [[d1,d2,1]]

            p_win = twodiceprob[(d1,d2)][0]
            win = max_move(op,turn+1,prob*p_win)

            if moves:
                op = moves + [[d1,d2,0]]
            else:
                op = [[d1,d2,0]]

            p_loss = twodiceprob[(d2,d1)][0]
            loss = max_move(op,turn+1,prob*p_loss)

            score = p_win * win + p_loss * loss

            smallest = min(smallest, score) 
    
    moveset = get_moveset(moves + [[d1]])
    min_states[moveset] = [turn, smallest]
    return smallest

import time
start_time = time.time()
print(max_move([],1,1))
print(f"Time taken is {time.time()-start_time}")


0.531176681130787
Time taken is 0.39199280738830566


In [36]:
# Stores the game state at any point in the game

class GameState:
    
    def __init__(self, p, turn, history, number=None):
        self.moves = []
        self.p = p
        self.turn = turn
        self.history = history
        self.number = number
    
    def add_move(self, move):
        self.moves.append(move)
    
    def best_move(self):
        # If a terminal node
        if self.moves == []:
            moves = get_moveset(self.history)
            won = checkwhowon(moves)
            return won
        # Make sure it isn't a chance node
        if self.turn % 2 == 0 or type(self.number) != int:
            if self.turn % 2 == 0:
                res = min(self.moves, key=lambda k:k[0]).copy()
            else:
                res = max(self.moves, key=lambda k:k[0]).copy()
            return res
        return None
    
start = GameState(0.530984044,1,[])
stack = [start]

def p1(cur, stack):
    moves = cur.history
    for d in dice:
        new_move = moves + [[d]]
        hash_move = get_moveset(new_move)
        
        if hash_move in min_states:
            result = min_states[hash_move]
            
            game_state = GameState(result[1], result[0], new_move, d)
            
            cur.add_move([result[1], game_state, game_state.number])
            stack.append(game_state)
            
    return cur, stack


# op = moves + [[d1,d2,0]]

def p2(cur, stack):
    moves = cur.history
    for d in dice:
        
        # Removes most recent dice move, and replaces with both dice moves
        new_move = moves[:-1] + [[cur.number, d]]
        loss = moves[:-1] + [[cur.number, d, 0]]
        win = moves[:-1] + [[cur.number, d, 1]]

        hash_loss = get_moveset(loss)
        hash_win = get_moveset(win)
 
        if hash_loss in max_states:
            win_result = max_states[hash_win]
            loss_result = max_states[hash_loss]
            
            prob = twodiceprob[(cur.number, d)][0]
            win_chance = prob * win_result[1] + (1-prob)* loss_result[1]
            
            game_state = GameState(win_chance, win_result[0], new_move, d)
            game_state_win = GameState(win_result[1], win_result[0], win,'Win')
            game_state_loss = GameState(loss_result[1], loss_result[0], loss,'Loss')
            
            # Adds in prob of happening
            game_state.add_move([prob, game_state_win, 'win'])
            game_state.add_move([1-prob, game_state_loss, 'loss'])
            
            stack.append(game_state_win)
            stack.append(game_state_loss)
            
            cur.add_move([win_chance, game_state, game_state.number])
    return cur, stack

# print(stack)
while stack:
#     print(stack)
    cur = stack.pop()
    if cur.turn % 2 == 0:
        cur, stack = p2(cur, stack)
    else:
        cur, stack = p1(cur, stack)
            
print(start)

In [39]:
start.best_move()[1].best_move()

[0.531176681130787, <__main__.GameState at 0x259eef4bef0>, 6]

In [35]:
test = start
while(test):
    print("")
    print(f"Num: {test.number}")
    print(f"Hist: {test.history}")
    print(f"Turn: {test.turn}")
    if test.best_move():
        print(f"Best move has prob of winning: {test.best_move()[0]} with a number of {test.best_move()[2]}")
    else:
        print(test.best_move())
    print('')
    if test.moves:
        out = sorted(test.moves,key = lambda k:k[0])
        for x in out:
            print(f"Move has prob: {x[0]} with a number of {x[2]}")
        print("")
        test = test.moves[1][1]
    else:
        test = False



Num: None
Hist: []
Turn: 1
0



In [40]:
import sys
print(f"size is {sys.getsizeof(max_states)/(1024**2)} MBs")
# len(max_states)

size is 5.000091552734375 MBs


In [6]:
def get_right(k):
    return [k[1][0], k[1][1]]

li = sorted(max_states.items(), key= lambda k:get_right(k))
li2 = []
for x in li:
    if len(x[0]) == 1 and x[0][0][0] == 4:
        li2.append(x)
li2.sort()
for x in li2:
    print(x)
for i in range(len(li2)//2):
    d_prob = twodiceprob[(li2[i*2][0][0][:2])][0]
    loss = (1 - d_prob) * li2[i*2][1][1]
    success = d_prob * li2[i*2+1][1][1]
    prob_of_winning = loss + success
    print("win: ", li2[i*2][1][1])
    print("loss: ",li2[i*2+1][1][1])
    print(prob_of_winning)

(((4, 6, 0),), [3, 0.382810625])
(((4, 6, 1),), [3, 0.6371538889])
(((4, 8, 0),), [3, 0.3451388611])
(((4, 8, 1),), [3, 0.6371538889])
(((4, 10, 0),), [3, 0.302603125])
(((4, 10, 1),), [3, 0.64843625])
(((4, 12, 0),), [3, 0.2624985])
(((4, 12, 1),), [3, 0.6791685000000001])
(((4, 20, 0),), [3, 0.07291625])
(((4, 20, 1),), [3, 0.6371538889])
win:  0.382810625
loss:  0.6371538889
0.531176681130787
win:  0.3451388611
loss:  0.6371538889
0.5458991927125001
win:  0.302603125
loss:  0.64843625
0.5619779687500001
win:  0.2624985
loss:  0.6791685000000001
0.5923636389
win:  0.07291625
loss:  0.6371538889
0.5666241840374999


In [7]:
li[-1]

(((20, 12, 0), (12, 20, 1), (20, 12, 1)), [7, 1])

In [8]:
def get_right(k):
    return [k[1][0], -k[1][1]]

li = sorted(min_states.items(), key= lambda k:get_right(k))
# li[0][0]
li[-1]

(((20, 12, 0), (12, 20, 0), (20,)), [6, 0.0])

In [9]:
start.best_move()[1].best_move()[1].moves[1][1].best_move()[1].best_move()[1].moves[1][1].best_move()

[0.125, <__main__.GameState at 0x25019c5a630>, 20]

In [ ]:
cur = start
while(cur.moves):
    print(cur.history)
    for x in cur.moves:
        print(x)
    print("Enter which one")
    new_move = int(input())
    cur = cur.moves[new_move][1]

In [10]:
import numpy as np
cur = start

def find_index(num):
    # Gets array of correlated numbers
    mov = np.array(cur.moves)[:,2]
    return int(np.where(mov==num)[0])
    
    


while(cur.moves):
    # Your move
    print(cur.history)
    poss_moves = [x[2] for x in cur.moves]
    print(f"\nYour possible moves are {poss_moves}")
#     for x in cur.moves:
#         print(x)
    print("Enter which one!\n")
    new_move = int(input())
    ind = find_index(new_move)
    cur = cur.moves[ind][1]
    
    #Enemy move
    best_move = cur.best_move()
    print(f"P2 is playing {best_move[2]} which leads to your {round(best_move[0],4)} chance of winning")
    cur = best_move[1]
    
    # Roll the dice
    # Gets the last 2 dice played
    d1 = cur.history[-1][0]
    d2 = cur.history[-1][1]
    ran1 = np.random.randint(1,d1+1)
    ran2 = np.random.randint(1,d2+1)
    print(f"Using a {d1} you got {ran1}, P2 used a {d2} and got {ran2}\n")
    won = True
    
    
    if ran2 < ran1:
        won = False
    elif ran2 == ran1 and d1 < d2:
        won = False

    if won:
        # Win is first in moves
        cur = cur.moves[0][1]
        print("You won")
    else:
        # Win is second in moves
        cur = cur.moves[1][1]
        print("You lost")
print(cur.best_move())
    

[]

Your possible moves are [4, 6, 8, 10, 12, 20]
Enter which one!

4
P2 is playing 6 which leads to your 0.5312 chance of winning
Using a 4 you got 1, P2 used a 6 and got 5

You won
[[4, 6, 1]]

Your possible moves are [6, 8, 10, 12, 20]
Enter which one!

6
P2 is playing 8 which leads to your 0.5586 chance of winning
Using a 6 you got 3, P2 used a 8 and got 5

You won
[[4, 6, 1], [6, 8, 1]]

Your possible moves are [4, 8, 10, 12, 20]
Enter which one!

4
P2 is playing 10 which leads to your 0.75 chance of winning
Using a 4 you got 1, P2 used a 10 and got 9

You won
Player 2 Wins


In [56]:
# start.best_move()[1].best_move()[1].moves[1][1].best_move()[1].best_move()[1].moves[1][1].best_move()
def game_test(cur):
    best = cur.best_move()
#     print(best)
    if best == 1:
        print(cur.history)
        return 1
    elif best == 0:
        print(cur.history)
        return 0
    elif best == None:
        d1 = cur.history[-1][0]
        d2 = cur.history[-1][1]
        ran1 = np.random.randint(1,d1+1)
        ran2 = np.random.randint(1,d2+1)
        
        won = True
        if ran2 < ran1:
            won = False
        elif ran2 == ran1 and d1 > d2:
            won = False

        if won:
            # Win is first in moves
            cur = cur.moves[0][1]
            return game_test(cur)
        else:
            # Win is second in moves
            cur = cur.moves[1][1]
            return game_test(cur)
    
    return game_test(best[1])

for i in range(20):
    game_test(start)

# total = 500000
# wins = 0
# for i in range(total):
#     wins += game_test(start)
#     if i % 10000 == 0 and i > 0:
#         prob_of_winning = wins/i
#         print(prob_of_winning)


[[4, 6, 1], [10, 12, 1], [20, 4, 0]]
[[4, 6, 1], [10, 12, 0], [12, 4, 0]]
[[4, 6, 0], [6, 8, 1], [4, 6, 1]]
[[4, 6, 1], [10, 12, 1], [20, 4, 0]]
[[4, 6, 1], [10, 12, 0], [12, 4, 0]]
[[4, 6, 0], [6, 8, 1], [4, 6, 0]]
[[4, 6, 0], [6, 8, 0], [20, 4, 0]]
[[4, 6, 0], [6, 8, 0], [20, 4, 0]]
[[4, 6, 1], [10, 12, 1], [20, 4, 0]]
[[4, 6, 0], [6, 8, 1], [4, 6, 1]]
[[4, 6, 1], [10, 12, 0], [12, 4, 0]]
[[4, 6, 1], [10, 12, 0], [12, 4, 0]]
[[4, 6, 1], [10, 12, 1], [20, 4, 0]]
[[4, 6, 0], [6, 8, 0], [20, 4, 1]]
[[4, 6, 1], [10, 12, 1], [20, 4, 0]]
[[4, 6, 1], [10, 12, 1], [20, 4, 1]]
[[4, 6, 0], [6, 8, 0], [20, 4, 0]]
[[4, 6, 0], [6, 8, 0], [20, 4, 0]]
[[4, 6, 1], [10, 12, 0], [12, 4, 0]]
[[4, 6, 1], [10, 12, 0], [12, 4, 0]]


In [83]:
def all_perfect_options(cur):
    best = cur.best_move()
    if best == 1:
        print(cur.history)
        return 1
    elif best == 0:
        print(cur.history)
        return 0
    elif best == None:
        temp = cur.moves[0][1]
        all_perfect_options(temp)
        temp = cur.moves[1][1]
        all_perfect_options(temp)
        return
        
    all_perfect_options(best[1])

all_perfect_options(start)


[[4, 6, 1], [10, 12, 1], [20, 4, 1]]
[[4, 6, 1], [10, 12, 1], [20, 4, 0]]
[[4, 6, 1], [10, 12, 0], [12, 4, 1]]
[[4, 6, 1], [10, 12, 0], [12, 4, 0]]
[[4, 6, 0], [6, 8, 1], [4, 6, 1]]
[[4, 6, 0], [6, 8, 1], [4, 6, 0]]
[[4, 6, 0], [6, 8, 0], [20, 4, 1]]
[[4, 6, 0], [6, 8, 0], [20, 4, 0]]
